# Basic cartoframes functionality

In [1]:
%matplotlib inline
import cartoframes
from cartoframes import Credentials
import pandas as pd

USERNAME = 'eschbacher' # <-- replace with your username 
APIKEY = 'abcdefghijklmnopqrstuvwxyz' # <-- your CARTO API key
creds = Credentials(username=USERNAME, 
                    key=APIKEY)
cc = cartoframes.CartoContext(creds=creds)

In [2]:
# Get a CARTO table as a pandas DataFrame
df = cc.read('brooklyn_poverty')
df.head()

,the_geom,the_geom_webmercator,geoid,poverty_count,total_population,poverty_per_pop,total_pop_2011_2015,pop_determined_poverty_status_2011_2015,commuters_16_over_2011_2015,walked_to_work_2011_2015
cartodb_id,,,,,,,,,,
2,0103000020E6100000010000000B0000006D3A02B85982...,0103000020110F0000010000000B000000D240DAA89070...,360470050003,23.112583,741.0,0.031191,9624.365242,3304.439797,4074.192637,0.005207
3,0103000020E610000001000000090000006AA164726A80...,0103000020110F000001000000090000009188EE63476D...,360470108003,818.804124,2067.0,0.396132,35743.802143,28261.979968,16316.285548,0.016649
4,0103000020E6100000010000000E000000591822A7AF7F...,0103000020110F0000010000000E000000F2591B1A0A6C...,360470110001,237.985714,1306.0,0.182225,12959.090059,17101.367690,4429.434939,0.021986
6,0103000020E61000000100000008000000F5BD86E0B87E...,0103000020110F000001000000080000009110AEED666A...,360470129012,57.217391,1143.0,0.050059,26191.505928,18825.265007,16044.550829,0.009637
7,0103000020E6100000010000000C000000E6CDE15AED7E...,0103000020110F0000010000000C0000008FF54811C06A...,360470169003,0.000000,572.0,0.000000,11023.184048,22815.245110,7607.777537,0.010880


In [3]:
from cartoframes import Layer, styling
l = Layer('brooklyn_poverty', color={'column': 'poverty_per_pop', 'scheme': styling.sunset(7)})
cc.map(layers=l, interactive=False)

In [4]:
# read in a CSV of NYC taxi data
df = pd.read_csv('taxi_sample.csv')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-05-01 14:52:11,2016-05-01 15:00:36,2,2.08,-74.006706,40.730461,1.0,N,-74.012383,40.706779,1.0,8.5,0.0,0.5,1.00,0.0,0.3,10.30
1,1,2016-05-01 08:34:08,2016-05-01 08:49:02,1,3.00,-73.924957,40.744125,1.0,N,-73.973824,40.762779,1.0,13.5,0.0,0.5,2.00,0.0,0.3,16.30
2,1,2016-05-04 09:44:40,2016-05-04 10:07:09,1,2.10,-73.973488,40.748501,1.0,N,-73.998955,40.740833,2.0,14.5,0.0,0.5,0.00,0.0,0.3,15.30
3,2,2016-05-01 20:50:11,2016-05-01 21:05:24,1,4.41,-73.999786,40.743267,1.0,N,-73.966362,40.792370,2.0,15.0,0.5,0.5,0.00,0.0,0.3,16.30
4,2,2016-05-02 07:26:56,2016-05-02 07:53:53,2,4.01,-73.963631,40.803360,1.0,N,-73.956963,40.784939,1.0,19.5,0.0,0.5,4.06,0.0,0.3,24.36


In [5]:
# send it to carto so we can map it
cc.write(df, 'taxi_sample',
         lnglat=('pickup_longitude', 'pickup_latitude'),
         overwrite=True)
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-05-01 14:52:11,2016-05-01 15:00:36,2,2.08,-74.006706,40.730461,1.0,N,-74.012383,40.706779,1.0,8.5,0.0,0.5,1.00,0.0,0.3,10.30
1,1,2016-05-01 08:34:08,2016-05-01 08:49:02,1,3.00,-73.924957,40.744125,1.0,N,-73.973824,40.762779,1.0,13.5,0.0,0.5,2.00,0.0,0.3,16.30
2,1,2016-05-04 09:44:40,2016-05-04 10:07:09,1,2.10,-73.973488,40.748501,1.0,N,-73.998955,40.740833,2.0,14.5,0.0,0.5,0.00,0.0,0.3,15.30
3,2,2016-05-01 20:50:11,2016-05-01 21:05:24,1,4.41,-73.999786,40.743267,1.0,N,-73.966362,40.792370,2.0,15.0,0.5,0.5,0.00,0.0,0.3,16.30
4,2,2016-05-02 07:26:56,2016-05-02 07:53:53,2,4.01,-73.963631,40.803360,1.0,N,-73.956963,40.784939,1.0,19.5,0.0,0.5,4.06,0.0,0.3,24.36


In [6]:
from cartoframes import Layer
cc.map(layers=Layer('taxi_sample'),
       interactive=False)

Oops, there are some zero-valued long/lats in there, so the results are going to [null island](https://en.wikipedia.org/wiki/Null_Island). Let's remove them.

In [7]:
df = df[(df['pickup_longitude'] != 0) | (df['pickup_latitude'] != 0)]
cc.write(df, 'taxi_sample', overwrite=True,
         lnglat=('pickup_longitude', 'pickup_latitude'))

In [8]:
# Let's take a look at what's going on at JFK airport, styled by the fare amount
cc.map(layers=Layer('taxi_sample',
                    size=4,
                    color={'column': 'fare_amount',
                           'scheme': styling.sunset(7)}),
       zoom=12, lng=-73.7880, lat=40.6629,
       interactive=False)